# RL PROJECT 

### STUDENT NAME & ROLL NO :
- Keerti (21CSU260)
- Aditya Sindhu(21CSU278) 
- Sowmya Sri(21CSU326)
- Snigdha (21CSU283)
- Abhishek Dubey(21CSU323) 


Semester: 5th 
Group: AIML-B (A3)


## PROBLEM STATEMENT 

Design an RL-based system for personalized dietary planning and exercise recommendations ." The objective is to develop an RL agent that tailors dietary plans and exercise routines for individuals with diabetes, taking into account their specific health data, dietary preferences, and historical records, to optimize glycaemic control and overall health.


In [1]:
import pandas as pd
import random 
dataset = pd.read_csv('diabetes.csv')

In [2]:
dataset

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
class Environment:
    def __init__(self, dataset):
        self.dataset = dataset

        # Initialize the state
        self.state = self.reset()

    def reset(self):
        if self is None:
            return None

        # Randomly select an individual from the dataset
        individual = random.choice(self.dataset)

        # Initialize the state
        state = individual[['blood_glucose_level', 'dietary_intake', 'physical_activity_level']]

        return state

    def step(self, action):
        # Take the action
        diet, exercise = action

        # Update the state
        next_state = self.update_state(diet, exercise)

        # Calculate the reward
        reward = self.calculate_reward(next_state)

        return next_state, reward, False

    def update_state(self, diet, exercise):
        # Update the individual's blood glucose level, dietary intake, and physical activity level based on the diet and exercise choices
        next_state = self.state.copy()

        # Update the blood glucose level
        next_state[0] += (diet == 'low_carb') - (diet == 'high_carb')

        # Update the dietary intake
        next_state[1] += (exercise == 'active') - (exercise == 'sedentary')

        # Update the physical activity level
        next_state[2] += (diet == 'low_carb') - (diet == 'high_carb')

        return next_state

    def calculate_reward(self, next_state):
        # Calculate the reward based on the next state
        reward = - (next_state[0] - 5.5)**2

        return reward



In [4]:
class RLAgent:
    def __init__(self, state_space, action_space, reward_function):
        self.state_space = state_space
        self.action_space = action_space
        self.reward_function = reward_function

        # Initialize the Q-table
        self.Q_table = np.zeros((state_space.shape[0], action_space.shape[0]))

    def act(self, state):
        """
        Selects an action based on the current state.

        Args:
            state: A vector of features representing the current state.

        Returns:
            An action from the action space.
        """

        # Epsilon-greedy exploration
        epsilon = 0.1
        if random.random() < epsilon:
            action = random.choice(self.action_space)
        else:
            action = np.argmax(self.Q_table[state])

        return action

    def learn(self, state, action, reward, next_state):
        """
        Updates the Q-table based on the current experience.

        Args:
            state: A vector of features representing the current state.
            action: The action that was taken.
            reward: The reward that was received.
            next_state: A vector of features representing the next state.
        """

        alpha = 0.1
        gamma = 0.9

        # Update the Q-table
        self.Q_table[state, action] += alpha * (reward + gamma * np.max(self.Q_table[next_state]))



In [5]:
def train_agent(agent, environment, n_episodes):
    for episode in range(n_episodes):
        state = environment.reset()

        while True:
            action = agent.act(state)

            next_state, reward, done = environment.step(action)

            agent.learn(state, action, reward, next_state)

            state = next_state

            if done:
                break


In [6]:
def deploy_agent(agent):
    # Load the dataset
    dataset = pd.read_csv('diabetes.csv')

    # Initialize the environment
    environment = Environment(dataset)

    # Get the individual's state
    state = environment.reset()

    # Generate the personalized dietary plan and exercise recommendations
    diet, exercise = agent.act(state)

    # Print the personalized dietary plan and exercise recommendations
    print('Dietary plan:', diet)
    print('Exercise recommendations:', exercise)


In [7]:
def get_diet_and_exercise(glucose_level, bmi):
    # Function to map health states to dietary plans and exercise recommendations based on the dataset

    # Define thresholds or ranges for health metrics based on the dataset characteristics
    glucose_threshold = 140  # threshold for high blood glucose
    bmi_threshold = 30  # threshold for high BMI

    # Determine the health state based on thresholds or ranges and provide recommendations
    if glucose_level > glucose_threshold and bmi > bmi_threshold:
        # Scenario 1: Elevated blood glucose and higher BMI
        # Recommendation for Scenario 1
        diet = 'Controlled carbohydrate intake with emphasis on complex carbs, fiber-rich foods, and portion control.'
        exercise = 'Increased physical activity to aid in weight management and improve insulin sensitivity. Regular aerobic exercises like brisk walking, cycling, or swimming for at least 30 minutes a day. Incorporation of strength training exercises to build muscle mass and boost metabolism.'
    elif glucose_level <= glucose_threshold and bmi <= bmi_threshold:
        # Scenario 2: Within desirable ranges for blood glucose and BMI
        # Recommendation for Scenario 2
        diet = 'Maintain a balanced diet for continued health and weight management. Emphasis on a diverse range of nutrients from various food groups. Portion control and moderation in consuming high-calorie or processed foods. Focus on fruits, vegetables, whole grains, lean protein sources, and healthy fats.'
        exercise = 'Sustained moderate physical activity for overall health maintenance. Regular moderate exercises such as walking, light jogging, or recreational sports for at least 30 minutes most days of the week. Engaging in activities that promote flexibility, balance, and cardiovascular health.'
    else:
        # Default recommendation if conditions don't match predefined thresholds
        diet = 'Balanced diet'
        exercise = 'Moderate exercise'

    return diet, exercise

# User input for individual's health metrics
glucose_input = float(input("Enter the individual's glucose level: "))
bmi_input = float(input("Enter the individual's BMI: "))
# Get diet and exercise recommendations based on the user-inputted health metrics
diet_recommendation, exercise_recommendation = get_diet_and_exercise(glucose_input, bmi_input)


print('Dietary plan:', diet_recommendation)
print()
print('Exercise recommendations:', exercise_recommendation)


Enter the individual's glucose level: 160
Enter the individual's BMI: 60
Dietary plan: Controlled carbohydrate intake with emphasis on complex carbs, fiber-rich foods, and portion control.

Exercise recommendations: Increased physical activity to aid in weight management and improve insulin sensitivity. Regular aerobic exercises like brisk walking, cycling, or swimming for at least 30 minutes a day. Incorporation of strength training exercises to build muscle mass and boost metabolism.
